# Collective Agreement Virtual Assistant
LangChain, AzureChatOpenAI, OpenAIEmbedding

In [31]:
# Install software packages:
%pip install langchain-openai==0.2.0
%pip install python-dotenv==1.0.0
%pip install langchain==0.3.4
%pip install langchainhub==0.1.13
%pip install pypdf==3.17.0
%pip install tiktoken==0.5.1
%pip install chromadb
%pip install langchain-community==0.3.20
%pip install unstructured
%pip install python-docx
%pip install unstructured langchain-community


Note: you may need to restart the kernel to use updated packages.
  Using cached langchain-0.3.4-py3-none-any.whl.metadata (7.1 kB)
Using cached langchain-0.3.4-py3-none-any.whl (1.0 MB)
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.25
    Uninstalling langchain-0.3.25:
      Successfully uninstalled langchain-0.3.25
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.3.20 requires langchain<1.0.0,>=0.3.21, but you have langchain 0.3.4 which is incompatible.


Note: you may need to restart the kernel to use updated packages.
  Using cached pypdf-3.17.0-py3-none-any.whl.metadata (7.5 kB)
Using cached pypdf-3.17.0-py3-none-any.whl (277 kB)
  Attempting uninstall: pypdf
    Found existing installation: pypdf 5.5.0
    Uninstalling pypdf-5.5.0:
      Successfully uninstalled pypdf-5.5.0
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unstructured-client 0.34.0 requires pypdf>=4.0, but you have pypdf 3.17.0 which is incompatible.


  Using cached tiktoken-0.5.1.tar.gz (32 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build tiktoken
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Building wheel for tiktoken (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [50 lines of output]
      C:\Users\gscgo\AppData\Local\Temp\pip-build-env-ufjtrn_0\overlay\Lib\site-packages\setuptools\config\_apply_pyprojecttoml.py:82: SetuptoolsDeprecationWarning: `project.license` as a TOML table is deprecated
      !!
      
              ********************************************************************************
              Please use a simple string containing a SPDX expression for `project.license`. You can also use `project.license-files`. (Both options available on setuptools>=77.0.0).
      
              By 2026-Feb-18, you need to update your project and remove deprecated calls
              or your builds will no longer be supported.
      
              See https://packaging.python.org/en/latest/guides/writing-pyproject-toml/#license for details.
              ************************************************

  Using cached uvicorn-0.34.2-py3-none-any.whl.metadata (6.5 kB)
  Using cached posthog-4.0.1-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached onnxruntime-1.22.0-cp312-cp312-win_amd64.whl.metadata (5.0 kB)
  Using cached opentelemetry_api-1.33.1-py3-none-any.whl.metadata (1.6 kB)
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.33.1-py3-none-any.whl.metadata (2.5 kB)
  Using cached opentelemetry_sdk-1.33.1-py3-none-any.whl.metadata (1.6 kB)
  Using cached tokenizers-0.21.1-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
  Using cached pypika-0.48.9-py2.py3-none-any.whl
  Using cached importlib_resources-6.5.2-py3-none-any.whl.metadata (3.9 kB)
  Using cached grpcio-1.71.0-cp312-cp312-win_amd64.whl.metadata (4.0 kB)
  Using cached bcrypt-4.3.0-cp39-abi3-win_amd64.whl.metadata (10 kB)
  Using cached kubernetes-32.0.1-py2.py3-none-any.whl.metadata (1.7 kB)
  Using cached google_auth-2.40.1-py2.py3-none-any.whl.metadata (6.2 kB)
  Using cached requests_oauthlib-2.0.0-py2.py3-none-any

In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings

os.environ["ALLOW_RESET"] = "TRUE"

load_dotenv('.env', override=True)

# # Explicitly set the deployment name to match your Azure deployment
embedding_model = AzureOpenAIEmbeddings(
    model=os.getenv('AZURE_OPENAI_EMBEDDING_DEPLOYMENT'),
    azure_endpoint=os.getenv('AZURE_OPENAI_EMBEDDING_ENDPOINT'),
    api_version=os.getenv('AZURE_OPENAI_EMBEDDING_API_VERSION'),
    api_key=os.getenv('AZURE_OPENAI_EMBEDDING_API_KEY')
    )

# Instantiate an AzureOpenAI LLM:
llm = AzureChatOpenAI(
    azure_deployment=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [3]:
#load web page from MOM PWM URLS:
from langchain.document_loaders import WebBaseLoader
loader_multiple_pages = WebBaseLoader(["https://www.mom.gov.sg/employment-practices/progressive-wage-model/cleaning-sector",
                                      "https://www.mom.gov.sg/employment-practices/progressive-wage-model/retail-sector",
                                      "https://www.mom.gov.sg/employment-practices/progressive-wage-model/food-services-sector",
                                      "https://www.mom.gov.sg/employment-practices/progressive-wage-model/occupational-pws-for-administrators-and-drivers",
                                      "https://www.mom.gov.sg/passes-and-permits/work-permit-for-foreign-worker/sector-specific-rules/non-traditional-sources-occupation-list"
                                      ])

## List of all pdf files (Tripartite Guidelines and)
from langchain.document_loaders import PyPDFDirectoryLoader
loader = PyPDFDirectoryLoader("./Tripartite Guidelines")

#load CAs in docx format
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import UnstructuredWordDocumentLoader

folder_path="./wst"
txt_loader = DirectoryLoader(folder_path, glob="*.docx", loader_cls=UnstructuredWordDocumentLoader)
txt_documents = txt_loader.load()

In [4]:
# Split content into chunks.
# Import a splitter module called RecursiveCharacterTextSplitter:
from langchain.text_splitter import RecursiveCharacterTextSplitter
# Instantiate the splitter:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
# Do the splitting:
splits = text_splitter.split_documents(loader.load())

# Embed and store splits in Chroma database.
# Import the Chroma and embeddings modules:
from langchain.vectorstores import Chroma
# from langchain.embeddings import OpenAIEmbeddings
# Instantiate the Chroma vector store with arguments:

# Comment out the following lines to process in smaller batches with delay to avoid hitting the rate limit when creating the embeddings:
# vectorstore = Chroma.from_documents(
#     documents=splits,
#     embedding=embedding_model
# )


import time
from tqdm import tqdm

# Create an empty Chroma DB first
vectorstore = Chroma(
    embedding_function=embedding_model
)

# Process in smaller batches
batch_size = 10
for i in tqdm(range(0, len(splits), batch_size)):
    batch = splits[i:i+batch_size]
    try:
        vectorstore.add_documents(documents=batch)
    except Exception as e:
        print(f"Error on batch {i}-{i+batch_size}: {e}")
        # Wait before retrying
        time.sleep(60)
        vectorstore.add_documents(documents=batch)
    # Small delay between successful batches
    time.sleep(2)

# Instantiate a retriever based on the vector store:
retriever = vectorstore.as_retriever() 




/var/folders/33/k25zs35d7w1dz5qh33sxsygw0000gn/T/ipykernel_10456/1449251159.py:26: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(
100%|██████████| 137/137 [06:17<00:00,  2.75s/it]


In [5]:
from langchain.chains import RetrievalQA

# Set up a retrieval chain:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(
        search_type="mmr",
        search_kwargs={"k": 5}
    ),
    return_source_documents=True
)

query = "In the event that retrenchment is inevitable, what should companies do?"

# Run the retrieval chain and get a response from the LLM:
completion = qa.invoke({"query": query})

# Display the completion:
print(completion['result'])

In the event that retrenchment is inevitable, companies should:

1. Ensure that retrenchment is the last resort, after considering and exhausting other feasible options.
2. Explain the company’s efforts to manage business challenges and the situation that led to the need for retrenchment.
3. Outline how the retrenchment exercise will be carried out and elaborate on the factors that will be considered.
4. Specify the assistance being offered to those affected, including potential counselling support.
5. If the company is unionised, notify the relevant union(s) early, ideally one month before notifying affected employees, and work collaboratively with the union to support affected employees.
6. Treat employees with respect during the retrenchment process, ensuring that it is carried out in a responsible and sensitive manner.
7. Consider implementing no-pay leave as an alternative, only after other measures have been considered and in consultation with unions and employees.


In [6]:
# Print each source in turn
print("\nSources:\n")
for idx, doc in enumerate(completion.get("source_documents", []), start=1):
    print(f"Source {idx}:\n{doc.page_content}\n\n-----\n")


Sources:

Source 1:
retrenchment1 should always be the last resort, after other feasible options have 
been considered and exhausted. In this way, employers will retai n their core 
capabilities, inspire loyalty in their workforce, and be well-positioned to emerge 
stronger in the recovery.  
 
3. Employers should press on with business and workforce transformation. 
This includes planning ahead and providing employees with resk illing, training

-----

Source 2:
• Explaining the company’s efforts to manage business challenges, and the 
business situation faced by the company resulting i n the need for a 
retrenchment exercise 
• Outlining how the retrenchment exercise will be carried out 
• Elaborating on the factors that will be considered 
• Specifying the assistance being offered to those affected 
 
14. Employers must do their part to ensure that the retrenchment is carried out

-----

Source 3:
implemented the necessary cost-saving measures, the tripartite consensus is that 
com